In [3]:
import pandas as pd
import glob
import regex as re
import query_gen as qg

In [4]:
files = glob.glob("results/*.csv")
dfs = [pd.read_csv(f, index_col=0) for f in files]
files

['results/eval_results_UniProt_oneshot_NousResearch-Hermes-3-Llama-3.2-3B-GGUF.csv',
 'results/eval_results_DBpedia_oneshot_NousResearch-Hermes-3-Llama-3.1-8B-GGUF.csv',
 'results/eval_results_BTO_oneshot_NousResearch-Hermes-3-Llama-3.1-8B-GGUF.csv',
 'results/eval_results_UniProt_oneshot_NousResearch-Hermes-3-Llama-3.1-8B-GGUF.csv',
 'results/eval_results_DBpedia_zeroshot_NousResearch-Hermes-3-Llama-3.2-3B-GGUF.csv',
 'results/eval_results_UniProt_zeroshot_NousResearch-Hermes-3-Llama-3.2-3B-GGUF.csv',
 'results/eval_results_BTO_oneshot_NousResearch-Hermes-3-Llama-3.2-3B-GGUF.csv',
 'results/eval_results_DBpedia_oneshot_NousResearch-Hermes-3-Llama-3.2-3B-GGUF.csv']

In [5]:

def graph_from_erl(erl: qg.EnrichedEntitiesRelations):
    G = qg.nx.DiGraph()
    for node in erl.entities:
        G.add_node(node.identifier, label=node.type)
    for link in erl.relations:
        G.add_edge(
            link.entity,
            link.target,
            weight=link.link.instance_count,
            label=link.relation,
        )
    return G

In [6]:
for f, df in zip(files, dfs):
    # data=re.findall(r"results/eval_results_(\w+)_.*", f)[0]
    # prompt_type=re.findall(r"results/eval_results_(\w+)_.*", f)[0]
    model = re.findall(r"results/eval_results_\w+_(.+)\.csv", f)[0]
    # df['prompt_type'] = prompt_type
    df["model"] = (
        model.replace("_", " ")
        .replace("-", " ")
        .rstrip("-GGUF")
        .lstrip("NousResearch")
    )

df = pd.concat(dfs)
df

,erl,response,generator,n_nodes,seed,erl_loaded,f1_score,precision,recall,ged,normed_ged,model,cfg_name,zeroshot
0,"{""relations"":[{""entity"":""Reviewed protein (rec...","{""relations"":[{""entity"":""protein 1"",""relation""...",llama,3,0,relations=[EnrichedRelation(entity='Reviewed p...,0.500000,1.0,0.333333,5.0,0.166667,Hermes 3 Llama 3.2 3B,UniProt,False
1,"{""relations"":[{""entity"":""Reviewed protein (rec...",a Reviewed protein (records) has a domain with...,templated,3,0,relations=[EnrichedRelation(entity='Reviewed p...,NaN,NaN,NaN,NaN,NaN,Hermes 3 Llama 3.2 3B,NaN,NaN
2,"{""relations"":[{""entity"":""Submission Citation"",...","{""relations"":[{""entity"":""submission"",""relation...",llama,3,1,relations=[EnrichedRelation(entity='Submission...,1.000000,1.0,1.000000,0.0,1.000000,Hermes 3 Llama 3.2 3B,UniProt,False
3,"{""relations"":[{""entity"":""Submission Citation"",...","{""relations"":[{""entity"":""Submission Citation 1...",templated,3,1,relations=[EnrichedRelation(entity='Submission...,1.000000,1.0,1.000000,0.0,1.000000,Hermes 3 Llama 3.2 3B,UniProt,False
4,"{""relations"":[{""entity"":""Citation"",""relation"":...",A citation is published in a journal and a rev...,llama,3,2,"relations=[EnrichedRelation(entity='Citation',...",NaN,NaN,NaN,NaN,NaN,Hermes 3 Llama 3.2 3B,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,"{""relations"":[{""entity"":""ethnic group"",""relati...",a ethnic group has a population place with pop...,templated,10,297,relations=[EnrichedRelation(entity='ethnic gro...,NaN,NaN,NaN,NaN,NaN,Hermes 3 Llama 3.2 3B,NaN,NaN
1796,"{""relations"":[{""entity"":""Organisation member"",...",An Organisation member has nationality of a co...,llama,10,298,relations=[EnrichedRelation(entity='Organisati...,NaN,NaN,NaN,NaN,NaN,Hermes 3 Llama 3.2 3B,NaN,NaN
1797,"{""relations"":[{""entity"":""Organisation member"",...",a Organisation member has a nationality with c...,templated,10,298,relations=[EnrichedRelation(entity='Organisati...,NaN,NaN,NaN,NaN,NaN,Hermes 3 Llama 3.2 3B,NaN,NaN
1798,"{""relations"":[{""entity"":""person"",""relation"":""b...",- A person is born in a populated place.\n- A ...,llama,10,299,"relations=[EnrichedRelation(entity='person', r...",NaN,NaN,NaN,NaN,NaN,Hermes 3 Llama 3.2 3B,NaN,NaN


In [7]:
df

,erl,response,generator,n_nodes,seed,erl_loaded,f1_score,precision,recall,ged,normed_ged,model,cfg_name,zeroshot
0,"{""relations"":[{""entity"":""Reviewed protein (rec...","{""relations"":[{""entity"":""protein 1"",""relation""...",llama,3,0,relations=[EnrichedRelation(entity='Reviewed p...,0.500000,1.0,0.333333,5.0,0.166667,Hermes 3 Llama 3.2 3B,UniProt,False
1,"{""relations"":[{""entity"":""Reviewed protein (rec...",a Reviewed protein (records) has a domain with...,templated,3,0,relations=[EnrichedRelation(entity='Reviewed p...,NaN,NaN,NaN,NaN,NaN,Hermes 3 Llama 3.2 3B,NaN,NaN
2,"{""relations"":[{""entity"":""Submission Citation"",...","{""relations"":[{""entity"":""submission"",""relation...",llama,3,1,relations=[EnrichedRelation(entity='Submission...,1.000000,1.0,1.000000,0.0,1.000000,Hermes 3 Llama 3.2 3B,UniProt,False
3,"{""relations"":[{""entity"":""Submission Citation"",...","{""relations"":[{""entity"":""Submission Citation 1...",templated,3,1,relations=[EnrichedRelation(entity='Submission...,1.000000,1.0,1.000000,0.0,1.000000,Hermes 3 Llama 3.2 3B,UniProt,False
4,"{""relations"":[{""entity"":""Citation"",""relation"":...",A citation is published in a journal and a rev...,llama,3,2,"relations=[EnrichedRelation(entity='Citation',...",NaN,NaN,NaN,NaN,NaN,Hermes 3 Llama 3.2 3B,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,"{""relations"":[{""entity"":""ethnic group"",""relati...",a ethnic group has a population place with pop...,templated,10,297,relations=[EnrichedRelation(entity='ethnic gro...,NaN,NaN,NaN,NaN,NaN,Hermes 3 Llama 3.2 3B,NaN,NaN
1796,"{""relations"":[{""entity"":""Organisation member"",...",An Organisation member has nationality of a co...,llama,10,298,relations=[EnrichedRelation(entity='Organisati...,NaN,NaN,NaN,NaN,NaN,Hermes 3 Llama 3.2 3B,NaN,NaN
1797,"{""relations"":[{""entity"":""Organisation member"",...",a Organisation member has a nationality with c...,templated,10,298,relations=[EnrichedRelation(entity='Organisati...,NaN,NaN,NaN,NaN,NaN,Hermes 3 Llama 3.2 3B,NaN,NaN
1798,"{""relations"":[{""entity"":""person"",""relation"":""b...",- A person is born in a populated place.\n- A ...,llama,10,299,"relations=[EnrichedRelation(entity='person', r...",NaN,NaN,NaN,NaN,NaN,Hermes 3 Llama 3.2 3B,NaN,NaN


In [8]:
df["prompt_type"] = df["zeroshot"].apply(lambda x: "zero shot" if x else "one shot")
df.drop(columns=["zeroshot"], inplace=True)
df

,erl,response,generator,n_nodes,seed,erl_loaded,f1_score,precision,recall,ged,normed_ged,model,cfg_name,prompt_type
0,"{""relations"":[{""entity"":""Reviewed protein (rec...","{""relations"":[{""entity"":""protein 1"",""relation""...",llama,3,0,relations=[EnrichedRelation(entity='Reviewed p...,0.500000,1.0,0.333333,5.0,0.166667,Hermes 3 Llama 3.2 3B,UniProt,one shot
1,"{""relations"":[{""entity"":""Reviewed protein (rec...",a Reviewed protein (records) has a domain with...,templated,3,0,relations=[EnrichedRelation(entity='Reviewed p...,NaN,NaN,NaN,NaN,NaN,Hermes 3 Llama 3.2 3B,NaN,zero shot
2,"{""relations"":[{""entity"":""Submission Citation"",...","{""relations"":[{""entity"":""submission"",""relation...",llama,3,1,relations=[EnrichedRelation(entity='Submission...,1.000000,1.0,1.000000,0.0,1.000000,Hermes 3 Llama 3.2 3B,UniProt,one shot
3,"{""relations"":[{""entity"":""Submission Citation"",...","{""relations"":[{""entity"":""Submission Citation 1...",templated,3,1,relations=[EnrichedRelation(entity='Submission...,1.000000,1.0,1.000000,0.0,1.000000,Hermes 3 Llama 3.2 3B,UniProt,one shot
4,"{""relations"":[{""entity"":""Citation"",""relation"":...",A citation is published in a journal and a rev...,llama,3,2,"relations=[EnrichedRelation(entity='Citation',...",NaN,NaN,NaN,NaN,NaN,Hermes 3 Llama 3.2 3B,NaN,zero shot
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,"{""relations"":[{""entity"":""ethnic group"",""relati...",a ethnic group has a population place with pop...,templated,10,297,relations=[EnrichedRelation(entity='ethnic gro...,NaN,NaN,NaN,NaN,NaN,Hermes 3 Llama 3.2 3B,NaN,zero shot
1796,"{""relations"":[{""entity"":""Organisation member"",...",An Organisation member has nationality of a co...,llama,10,298,relations=[EnrichedRelation(entity='Organisati...,NaN,NaN,NaN,NaN,NaN,Hermes 3 Llama 3.2 3B,NaN,zero shot
1797,"{""relations"":[{""entity"":""Organisation member"",...",a Organisation member has a nationality with c...,templated,10,298,relations=[EnrichedRelation(entity='Organisati...,NaN,NaN,NaN,NaN,NaN,Hermes 3 Llama 3.2 3B,NaN,zero shot
1798,"{""relations"":[{""entity"":""person"",""relation"":""b...",- A person is born in a populated place.\n- A ...,llama,10,299,"relations=[EnrichedRelation(entity='person', r...",NaN,NaN,NaN,NaN,NaN,Hermes 3 Llama 3.2 3B,NaN,zero shot


In [9]:
df_clean = df.dropna()
df_clean.describe()

,n_nodes,seed,f1_score,precision,recall,ged,normed_ged
count,8940.000000,8940.000000,8940.000000,8940.000000,8940.000000,8940.000000,8940.000000
mean,5.894966,149.045526,0.771089,0.867564,0.729105,3.813311,0.573046
std,2.908962,86.073028,0.239136,0.214251,0.270144,5.659379,0.337644
min,3.000000,0.000000,0.133333,0.200000,0.100000,0.000000,-2.666667
25%,3.000000,75.000000,0.500000,0.666667,0.500000,1.000000,0.300000
50%,5.000000,149.000000,0.800000,1.000000,0.750000,2.000000,0.600000
75%,10.000000,223.000000,1.000000,1.000000,1.000000,4.000000,0.888889
max,10.000000,299.000000,1.000000,1.000000,1.000000,61.000000,1.000000


In [10]:
df["erl_loaded"] = df["erl"].apply(qg.EnrichedEntitiesRelations.model_validate_json)
# df["response_loaded"] = df["response"].apply(lambda x: qg.EnrichedEntitiesRelations.model_validate_json(x) if isinstance(x, str) else None)
df["erl_graph"] = df["erl_loaded"].apply(graph_from_erl)
# df["response_graph"] = df["response"].apply(lambda x: graph_from_erl if x is not None else None)

In [11]:
# df["normed_ged"] = 1 - df["ged"] / df["erl_graph"].apply(
#     lambda x: (len(x.nodes) + len(x.nodes))
# )

In [12]:
means = df_clean.groupby(["n_nodes", "prompt_type", "model", "generator", "cfg_name"])[
    ["f1_score", "normed_ged"]
].mean()
counts = df_clean.groupby(["n_nodes", "prompt_type", "model", "generator", "cfg_name"])[
    ["f1_score", "normed_ged"]
].count()
means

f1_score  \
n_nodes prompt_type model                  generator cfg_name             
3       one shot    Hermes 3 Llama 3.1 8B  human     DBpedia   0.700000   
                                           llama     BTO       0.690041   
                                                     DBpedia   0.773518   
                                                     UniProt   0.709953   
                                           templated BTO       0.773091   
                                                     DBpedia   0.873156   
                                                     UniProt   0.817474   
                    Hermes 3 Llama 3.2 3B  human     DBpedia   0.688889   
                                           llama     BTO       0.766569   
                                                     DBpedia   0.792252   
                                                     UniProt   0.739227   
                                           templated BTO       0.944039   
                                                     DBpedia   0.879489   
                                                     UniProt   0.868221   
        zero shot   Hermes 3 Llama 3.2 3B  human     DBpedia   0.500000   
                                           llama     DBpedia   0.648958   
                                                     UniProt   0.627381   
                                           templated DBpedia   0.608712   
                                                     UniProt   0.774653   
5       one shot    Hermes 3 Llama 3.1 8B  human     DBpedia   0.933333   
                                           llama     BTO       0.683921   
                                                     DBpedia   0.733572   
                                                     UniProt   0.719127   
                                           templated BTO       0.798756   
                                                     DBpedia   0.845764   
                                                     UniProt   0.786174   
                    Hermes 3 Llama 3.2 3B  human     DBpedia   1.000000   
                                           llama     BTO       0.774851   
                                                     DBpedia   0.716179   
                                                     UniProt   0.746519   
                                           templated BTO       0.948344   
                                                     DBpedia   0.797523   
                                                     UniProt   0.878965   
        zero shot   Hermes 3 Llama 3.2 3B  human     DBpedia   0.500000   
                                           llama     DBpedia   0.563743   
                                                     UniProt   0.586640   
                                           templated DBpedia   0.566283   
                                                     UniProt   0.783247   
10      one shot    Hermes 3 Llama 3.1 8B  human     DBpedia   0.652288   
                                           llama     BTO       0.662936   
                                                     DBpedia   0.666543   
                                                     UniProt   0.699440   
                                           templated BTO       0.778662   
                                                     DBpedia   0.793400   
                                                     UniProt   0.786138   
                    Hermes 3 Llama 3.2 3B  human     DBpedia   0.583333   
                                           llama     BTO       0.767735   
                                                     DBpedia   0.608290   
                                                     UniProt   0.721885   
                                           templated BTO       0.955770   
                                                     DBpedia   0.762484   
                                                     UniProt   0.859680   
        zero 

In [13]:
counts

f1_score  \
n_nodes prompt_type model                  generator cfg_name             
3       one shot    Hermes 3 Llama 3.1 8B  human     DBpedia          2   
                                           llama     BTO            186   
                                                     DBpedia        213   
                                                     UniProt        222   
                                           templated BTO            212   
                                                     DBpedia        215   
                                                     UniProt        224   
                    Hermes 3 Llama 3.2 3B  human     DBpedia          3   
                                           llama     BTO            243   
                                                     DBpedia        244   
                                                     UniProt        223   
                                           templated BTO            274   
                                                     DBpedia        244   
                                                     UniProt        239   
        zero shot   Hermes 3 Llama 3.2 3B  human     DBpedia          1   
                                           llama     DBpedia         64   
                                                     UniProt        112   
                                           templated DBpedia         40   
                                                     UniProt         96   
5       one shot    Hermes 3 Llama 3.1 8B  human     DBpedia          3   
                                           llama     BTO            185   
                                                     DBpedia        208   
                                                     UniProt        225   
                                           templated BTO            211   
                                                     DBpedia        210   
                                                     UniProt        226   
                    Hermes 3 Llama 3.2 3B  human     DBpedia          3   
                                           llama     BTO            237   
                                                     DBpedia        230   
                                                     UniProt        223   
                                           templated BTO            279   
                                                     DBpedia        231   
                                                     UniProt        234   
        zero shot   Hermes 3 Llama 3.2 3B  human     DBpedia          2   
                                           llama     DBpedia         79   
                                                     UniProt        120   
                                           templated DBpedia         53   
                                                     UniProt        101   
10      one shot    Hermes 3 Llama 3.1 8B  human     DBpedia          3   
                                           llama     BTO            185   
                                                     DBpedia        181   
                                                     UniProt        233   
                                           templated BTO            209   
                                                     DBpedia        171   
                                                     UniProt        227   
                    Hermes 3 Llama 3.2 3B  human     DBpedia          3   
                                           llama     BTO            241   
                                                     DBpedia        169   
                                                     UniProt        223   
                                           templated BTO            273   
                                                     DBpedia        157   
                                                     UniProt        232   
        zero 

In [14]:
column_map = {
    "f1_score": "$F_1$",
    "normed_ged": "$GED_{s}$",
    "prompt_type": "Prompt Type",
    "model": "Model",
    "n_nodes": "$k$",
    "generator": "Query Origin",
}

## Compare Query Origins on DBpedia with 8B model

In [15]:
query_gen_compare = df_clean[
    df_clean["model"].str.contains("8B") & df_clean["cfg_name"].str.contains("DBpedia") & df_clean["prompt_type"].str.contains("one shot")
]
grouped_query_gen_compare = query_gen_compare.groupby(["n_nodes", "generator"])[
    ["f1_score", "normed_ged"]
].mean()
grouped_query_gen_compare = grouped_query_gen_compare.rename(
    columns=column_map
).rename_axis([column_map["n_nodes"], column_map["generator"]], axis=0)
with open("tables/compare_gen_query.tex", "w") as f:
    f.write(
        grouped_query_gen_compare.to_latex(
            float_format="%.2f",
            caption="Comparison of query generation methods for LLama 3.1 8B on DBpedia with one shot prompts.",
            label="tab:compare_gen_query",
        )
    )
grouped_query_gen_compare

$F_1$  $GED_{s}$
$k$ Query Origin                     
3   human         0.700000   0.500000
    llama         0.773518   0.564611
    templated     0.873156   0.737290
5   human         0.933333   0.700000
    llama         0.733572   0.442061
    templated     0.845764   0.640515
10  human         0.652288   0.232080
    llama         0.666543   0.291221
    templated     0.793400   0.570159

## Compare models across Datasets

In [21]:
datasets = ["DBpedia", "UniProt", "BTO"]
for dataset in datasets:
    model_compare = df_clean[df_clean["cfg_name"].str.strip() == dataset]
    grouped_model_compare = model_compare.groupby(["n_nodes", "prompt_type", "model"])[
        ["f1_score", "normed_ged"]
    ].mean()
    grouped_model_compare = grouped_model_compare.rename(
        columns=column_map
    ).rename_axis([column_map["n_nodes"], column_map["prompt_type"], column_map["model"]], axis=0)
    with open(f"tables/compare_model_{dataset}.tex", "w") as f:
        f.write(
            grouped_model_compare.to_latex(
                float_format="%.2f",
                caption=f"Comparison of models for {dataset}",
                label=f"tab:compare_model_{dataset}",
                # sparsify=False
            )
        )
grouped_model_compare

$F_1$  $GED_{s}$
$k$ Prompt Type Model                                      
3   one shot    Hermes 3 Llama 3.1 8B   0.734279   0.703455
                Hermes 3 Llama 3.2 3B   0.860624   0.542712
5   one shot    Hermes 3 Llama 3.1 8B   0.745109   0.728220
                Hermes 3 Llama 3.2 3B   0.868658   0.574960
10  one shot    Hermes 3 Llama 3.1 8B   0.724324   0.716283
                Hermes 3 Llama 3.2 3B   0.867606   0.557436

In [17]:
dataset

'BTO'

In [18]:
df_clean["cfg_name"].str.strip() == "DBpedia"

0       False
2       False
3       False
5       False
7       False
        ...  
1783     True
1785     True
1791     True
1792     True
1799     True
Name: cfg_name, Length: 8940, dtype: bool